In [ ]:
#!pip install pandas_gbq

In [34]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import sqlite3

In [35]:
json_credentials_path = r'C:\Users\hills\Documents\Fall2023\ADA\wedge-project\umt-msba-pin.json'
# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_credentials_path

# Initialize the BigQuery client
client = bigquery.Client()
credentials = service_account.Credentials.from_service_account_file(json_credentials_path)
gbq_proj_id = 'umt-msba'

for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [36]:
query1="""

SELECT
  DATE(datetime) AS formatted_date,
  EXTRACT(hour FROM TIMESTAMP(datetime)) AS hour,
  ROUND(SUM(total), 2) AS spend,
  COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no, trans_no))) AS trans,
  SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS Items
--FROM `umt-msba.transactions.transArchive_201001_201003`
FROM `umt-msba.transactions.transArchive_*`
WHERE department not in (0, 15) AND
(trans_status is null OR
trans_status in ("R","V", " "))
GROUP BY formatted_date, hour ORDER BY formatted_date, hour

"""

In [37]:
# Establishing a connection to the DB or create it if it doesn't exist
conn = sqlite3.connect("reporting.db")


In [38]:
holder = pandas_gbq.read_gbq(query1,project_id=gbq_proj_id,credentials=credentials)

Downloading: 100%|██████████|


In [39]:
holder.head()

,formatted_date,hour,spend,trans,Items
0,2010-01-01,9,1006.28,36,245
1,2010-01-01,10,3128.55,82,913
2,2010-01-01,11,4001.66,118,1108
3,2010-01-01,12,3886.51,124,1143
4,2010-01-01,13,4654.52,154,1365


In [40]:
holder.to_sql("date_hour",conn,if_exists="replace",index=False)


2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 


In [41]:
query2 = """
SELECT
  card_no,
  EXTRACT(year
  FROM
    TIMESTAMP(datetime)) AS year,
  EXTRACT(month
  FROM
    TIMESTAMP(datetime)) AS month,
  ROUND(SUM(total), 2) AS sales,
  COUNT(DISTINCT(CONCAT(EXTRACT(date
        FROM
          datetime),register_no,emp_no, trans_no))) AS trans,
  SUM(CASE
      WHEN trans_status IN ("R", "V") THEN -1
    ELSE
    1
  END
    ) AS Items
FROM
  `umt-msba.transactions.transArchive_*`
WHERE
  department NOT IN (0,
    15)
  AND (trans_status IS NULL
    OR trans_status IN ("R",
      "V",
      " "))
GROUP BY
  card_no,
  year,
  month
ORDER BY
  card_no,
  year,
  month
"""

In [42]:
sales_by_owner_by_year_by_month = pandas_gbq.read_gbq(query2, project_id=gbq_proj_id, credentials=credentials)

sales_by_owner_by_year_by_month.to_sql("sales_by_owner_by_year_by_month", conn, if_exists="replace", index=False)


Downloading: 100%|██████████|


3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [45]:
query3 = """
SELECT
  t.upc,
  t.description,
  t.department AS department_number,
  dl.dept_name AS department_name,
  EXTRACT(year FROM TIMESTAMP(t.datetime)) AS year,
  EXTRACT(month FROM TIMESTAMP(t.datetime)) AS month,
  ROUND(SUM(t.total), 2) AS sales,
  COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no, trans_no))) AS trans,
  SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS Items
FROM
  `umt-msba.transactions.transArchive_*` AS t
JOIN
  `umt-msba.transactions.department_lookup` AS dl
ON
  t.department = dl.department
WHERE
  t.department NOT IN (0,
    15)
  AND (trans_status IS NULL
    OR trans_status IN ("R",
      "V",
      " "))
GROUP BY
  t.upc, t.description, t.department, department_name, year, month
ORDER BY
  t.upc, year, month
"""


In [46]:
sales_by_product_description_by_year_by_month = pandas_gbq.read_gbq(query3, project_id=gbq_proj_id, credentials=credentials)

sales_by_product_description_by_year_by_month.to_sql("sales_by_product_description_by_year_by_month", conn, if_exists="replace", index=False)


Downloading: 100%|██████████|


In [47]:
#close last
conn.close()